# Relatório do Projeto Final

## Machine Learning - 2023.1

### Autores: Marco Moliterno e Renato Falcão

Este relatório tem o objetivo de descrever a aplicação de técnicas de aprendizado de máquina para prever o preço de cartas de Yu-Gi-Oh baseando-se nas múltiplas características das mesmas.

O dataset estudado é uma união da base de dados, disponibilizada no [Kaggle](https://www.kaggle.com/), com os preços médios das cartas, disponibilizada pela API do website [Yu-Gi-Oh Prices](https://yugiohprices.com/).

#### Referências

Base de dados do Kaggle disponível [aqui](https://www.kaggle.com/datasets/thedevastator/yu-gi-oh-dataset?select=yugioh_enriched.csv).

Requisições web realizadas ao iterar sobre os sets de cartas, obtendo os preços das cartas pertencentes aos respectivos sets. Esta API pode ser consumida pelo endpoint `https://yugiohprices.com/api/set_data/{nome_do_set}`. Exemplo de resposta pode ser observado [aqui](https://yugiohprices.com/api/set_data/2013%20Collectible%20Tins%20Wave%201).

## Inicio do projeto

### Importando bibliotecas e carregando base de dados

Para realizar o estudo, estaremos utilizando especialmente as bibliotecas *Pandas*, *MatPlotLib*, *Scikit-Learn* e *Numpy*. Vamos importá-las e carregar a base de dados com o Pandas.

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv("dbs/yugi_db_polished.csv", index_col=0)

df.head()

,Name,Rarity,Price,Description,CardType,Attribute,ATK,DEF,LVL,Property,...,isEffect,Ability1,Ability2,isTuner,isPendulum,PCA1,PCA2,PCA3,PCA4,PCA5
0,XX-Saber Boggart Knight,Shatterfoil Rare,2.73,When this card is Normal Summoned: You can Spe...,monster,earth,1900.0,1000.0,4.0,NaN,...,Effect,NaN,NaN,NaN,NaN,0.188272,0.247905,0.176880,0.404444,-0.121330
1,Gagaga Cowboy,Shatterfoil Rare,4.68,2 Level 4 monsters\r\n\r\nOnce per turn: You c...,monster,earth,1500.0,2400.0,4.0,NaN,...,Effect,Xyz,NaN,NaN,NaN,0.195065,-0.032889,-0.386652,0.147118,0.259788
2,Forbidden Chalice,Shatterfoil Rare,2.09,Target 1 face-up monster on the field; until t...,spell,NaN,NaN,NaN,NaN,Quick-Play,...,NaN,NaN,NaN,NaN,NaN,0.586491,-0.312442,-0.028247,0.186492,0.210409
3,Fairy Cheer Girl,Shatterfoil Rare,2.06,2 Level 4 Fairy-Type monsters\r\n\r\nYou can d...,monster,light,1900.0,1500.0,4.0,NaN,...,Effect,Xyz,NaN,NaN,NaN,0.240005,0.365704,-0.041211,0.027144,0.274765
4,Exploder Dragon,Shatterfoil Rare,2.08,If this card is destroyed by battle and sent t...,monster,earth,1000.0,0.0,3.0,NaN,...,Effect,NaN,NaN,NaN,NaN,0.335846,-0.862453,0.093023,-0.371786,0.023188


### Análise Exploratória e Limpeza dos Dados

Com os dados em mãos, podemos começar realizando uma análise exploratória na qual desejamos identificar as características do dataset, observando a estrutura dos dados disponíveis, presença de anomalias, tendências, padrões, distribuições e dados ausentes. O entendimento dos dados é fundamental para se poder tomar decisões com bom embasamento acerca da abordagem que utilizaremos.

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11574 entries, 0 to 11573
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         11574 non-null  object 
 1   Rarity       11574 non-null  object 
 2   Price        11574 non-null  float64
 3   Description  11574 non-null  object 
 4   CardType     11574 non-null  object 
 5   Attribute    7470 non-null   object 
 6   ATK          7580 non-null   float64
 7   DEF          7580 non-null   float64
 8   LVL          7580 non-null   float64
 9   Property     3994 non-null   object 
 10  MonsterType  7580 non-null   object 
 11  isEffect     7580 non-null   object 
 12  Ability1     1709 non-null   object 
 13  Ability2     138 non-null    object 
 14  isTuner      453 non-null    object 
 15  isPendulum   277 non-null    object 
 16  PCA1         11574 non-null  float64
 17  PCA2         11574 non-null  float64
 18  PCA3         11574 non-null  float64
 19  PCA4

In [12]:
print(df.columns)

Index(['Name', 'Rarity', 'Price', 'Description', 'CardType', 'Attribute',
       'ATK', 'DEF', 'LVL', 'Property', 'MonsterType', 'isEffect', 'Ability1',
       'Ability2', 'isTuner', 'isPendulum', 'PCA1', 'PCA2', 'PCA3', 'PCA4',
       'PCA5'],
      dtype='object')


Explicar o que significa cada coluna

In [6]:
df.head()

,Name,Rarity,Price,Description,CardType,Attribute,ATK,DEF,LVL,Property,...,isEffect,Ability1,Ability2,isTuner,isPendulum,PCA1,PCA2,PCA3,PCA4,PCA5
0,XX-Saber Boggart Knight,Shatterfoil Rare,2.73,When this card is Normal Summoned: You can Spe...,monster,earth,1900.0,1000.0,4.0,NaN,...,Effect,NaN,NaN,NaN,NaN,0.190090,0.246757,0.177431,0.413126,-0.107253
1,Gagaga Cowboy,Shatterfoil Rare,4.68,2 Level 4 monsters\r\n\r\nOnce per turn: You c...,monster,earth,1500.0,2400.0,4.0,NaN,...,Effect,Xyz,NaN,NaN,NaN,0.204763,-0.029506,-0.446906,0.106477,0.238566
2,Forbidden Chalice,Shatterfoil Rare,2.09,Target 1 face-up monster on the field; until t...,spell,NaN,NaN,NaN,NaN,Quick-Play,...,NaN,NaN,NaN,NaN,NaN,0.585765,-0.314684,-0.037465,0.173857,0.209305
3,Fairy Cheer Girl,Shatterfoil Rare,2.06,2 Level 4 Fairy-Type monsters\r\n\r\nYou can d...,monster,light,1900.0,1500.0,4.0,NaN,...,Effect,Xyz,NaN,NaN,NaN,0.258598,0.327643,-0.085230,-0.001137,0.275321
4,Exploder Dragon,Shatterfoil Rare,2.08,If this card is destroyed by battle and sent t...,monster,earth,1000.0,0.0,3.0,NaN,...,Effect,NaN,NaN,NaN,NaN,0.335081,-0.860984,0.093261,-0.375335,-0.007053


Agora, a ideia é realizar um pré-processamento desses dados ...

### Pré-processamento dos dados

Nesta etapa, os dados são transformados em uma forma adequada para serem utilizados pelos modelos de Machine Learning. Isso pode envolver a normalização ou padronização das variáveis, a codificação de variáveis categóricas, a redução de dimensionalidade e outras técnicas de preparação dos dados.

### Divisão dos dados

Antes de prosseguir para a seleção de um modelo e treinamento, é necessário dividir os dados em conjuntos de treinamento, validação e teste. O conjunto de treinamento é utilizado para treinar o modelo, o conjunto de validação é usado para ajustar os parâmetros do modelo e o conjunto de teste é utilizado para avaliar o desempenho final do modelo. Esta divisão é importante para evitar que a avaliação do modelo seja enviesada pelo resultado final, de forma a "overfittar" para os dados de teste. 

### Seleção e Treinamento do modelo

O modelo selecionado é o ... e será treinado usando o conjunto de treinamento. O treinamento envolve alimentar os dados ao modelo e ajustar seus parâmetros com o objetivo de aprender os padrões presentes nos dados.

### Avaliação do modelo

O desempenho do modelo é avaliado usando o conjunto de validação. Métricas como acurácia, precisão, recall, F1-score e curvas de aprendizado são utilizadas para avaliar o quão bem o modelo está generalizando os dados.

### Ajuste do modelo

Com base na avaliação do modelo, é possível ajustar os parâmetros, selecionar diferentes algoritmos ou fazer outras modificações para melhorar o desempenho do modelo.

### Teste do modelo

O modelo ajustado é então testado usando o conjunto de teste, que contém dados não vistos anteriormente. Isso permite avaliar como o modelo se comporta em situações reais e verificar sua capacidade de generalização.